In [ ]:
import bs4 as bs
import urllib. request
import time
from datetime import datetime
import pandas as pd
import json
import os
import pandas as pd

# Scraping all rental data 

# Scraping entire data may take over 18 hours

In [ ]:
def scrape_immobili(type_house,max_pages):
    for seite in range(1,max_pages):
    
        print("Loop " + str(seite) + " startet.")

        df = pd.DataFrame()
        l=[]

        try:

            soup = bs.BeautifulSoup(urllib.request.urlopen("https://www.immobilienscout24.de/Suche/S-2/P-"+str(seite)+"/" + type_house).read(),'lxml')
            print("Aktuelle Seite: "+"https://www.immobilienscout24.de/Suche/S-2/P-"+str(seite)+"/"+type_house)
            for paragraph in soup.find_all("a"):

                if r"/expose/" in str(paragraph.get("href")):
                    l.append(paragraph.get("href").split("#")[0])

                l = list(set(l))

            for item in l:

                try:

                    soup = bs.BeautifulSoup(urllib.request.urlopen('https://www.immobilienscout24.de'+item).read(),'lxml')

                    data = pd.DataFrame(json.loads(str(soup.find_all("script")).split("keyValues = ")[1].split("}")[0]+str("}")),index=[str(datetime.now())])

                    data["URL"] = str(item)

                    beschreibung = []

                    for i in soup.find_all("pre"):
                        beschreibung.append(i.text)

                    data["beschreibung"] = str(beschreibung)

                    df = df.append(data)

                except Exception as e: 
                    print(str(datetime.now())+": " + str(e))
                    l = list(filter(lambda x: x != item, l))
                    print("ID " + str(item) + " entfernt.")
            print("Exportiert CSV")
            df.to_csv(str(datetime.now())[:19].replace(":","").replace(".","")+".csv",sep=";",decimal=",",encoding = "utf-8",index_label="timestamp")     

            print("Loop " + str(seite) + " endet.\n")

        except Exception as e: 
            print(str(datetime.now())+": " + str(e))

    print("End of scraping!")

    df = pd.DataFrame()
    n = 0
    for i in os.listdir()[2:-2]:
        n = n + 1
        df = df.append(pd.read_csv(str(i),sep = ";" , decimal = "," , encoding = "utf-8" ))
        print ( "passage" + str ( n ) )
    
    df.to_csv(type_houseouse + '_data.csv')
    
    return df


# Scraping 4250 pages of wohnung-mieten (rent)
1. Make sure that entire directory is empty before running the scraping function every time

In [ ]:
scrape_immobili('wohnung-mieten',4250)

### clear the dirctory before running the next function

In [ ]:
scrape_immobili('haus-mieten',230)

### clear the dirctory before running the next function
# Scraping Purchase data

In [ ]:
scrape_immobili('wohnung-kaufen',1650)

### clear the dirctory before running the next function

In [ ]:
scrape_immobili('haus-kaufen',2300)

# Note for Professor
1. since entire scraping would take over 18hrs we decided to scrape different pages saperately and merge the data together at the end to save time

2. The number of pages that was available to scrape was initially estimated roughly based on the number of results that was shown, later we did a binary search to find exact number of pages, this wasen't done using any code so we haven't included it. Approximation used ~ (No of search results)/20 (as every page shows only 20 results) 